In [1]:
import networkx as nx
# create a DAG
g = nx.DiGraph()
# add nl[j]s and edges to the DAG
g.add_edges_from([("A", "D"), ("B", "D"), ("B", "E"), ("B", "G"), ("C", "E"),("C", "F"), ("D", "I"), ("E", "G"),("E", "H"), ("F", "H"), ("G", "I"), ("G","J"), ("H", "J")])
nx.set_node_attributes(g, {"A": "PI", "B": "PI", "C":"PI", "D":"AND2", "E":"XOR2", "F":"INV", "G":"AND2", "H":"AND2", "I":"OR2", "J":"OR2"},name="gateType")
print(g.nodes(data=True))
# sort the nodes in topological order
nl = list(nx.topological_sort(g))
print('Nodes in topological order',nl)
# find children of a node
print('Successors of node B', list(g.successors("B")))
# find parents of a node
print('Parents of node H', list(g.predecessors("H")))
['C', 'F', 'A', 'B', 'D', 'E', 'G', 'H', 'I', 'J']

[('A', {'gateType': 'PI'}), ('D', {'gateType': 'AND2'}), ('B', {'gateType': 'PI'}), ('E', {'gateType': 'XOR2'}), ('G', {'gateType': 'AND2'}), ('C', {'gateType': 'PI'}), ('F', {'gateType': 'INV'}), ('I', {'gateType': 'OR2'}), ('H', {'gateType': 'AND2'}), ('J', {'gateType': 'OR2'})]
Nodes in topological order ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
Successors of node B ['D', 'E', 'G']
Parents of node H ['E', 'F']


['C', 'F', 'A', 'B', 'D', 'E', 'G', 'H', 'I', 'J']

# Getting the input from the netlist file

In [1]:
# Importing the necessary libraries
import networkx as nx
import copy
# Declaring variables for opening the files required
netwrok = "c432.net"
inputs = "c432.inputs"

# Defining a class to take in the vlaues from the network file
class Component:
	def __init__(self,name,g_type,input,output):
		self.name = name
		self.g_type = g_type
		self.input = input
		self.output = output
                
# Declaring a list to store the components
l = []

# Opening the file mentioned and storing it in the list
with open(netwrok) as f:
    lines = f.readlines()
    for line in lines:
        name,g_type,*input= line.split()
        output = line.split()[-1]
        Gate = Component(name,g_type,input,output)
        l.append(Gate)

# Declaring and creating the  edges list for the topoligal sorting
edges = []
g = nx.DiGraph()
for gate in l:
    if len(gate.input) == 3:
        edges.append((gate.input[0],gate.output))
        edges.append((gate.input[1],gate.output))
    else:
        edges.append((gate.input[0],gate.output))

# Making the diGraph for the topological sorting
g.add_edges_from(edges)

# Taking inputs from the inputs file and storing them in the values list as dictionaries
value = []
with open(inputs) as file:
    lineIn = file.readlines()
    pi = lineIn[0].split()
    for i in range(len(lineIn)-1):
        value.append({})
        for j in range(len(pi)):
            value[i].update({lineIn[0].split()[j]:lineIn[i+1].split()[j]})
value1 = copy.copy(value)

# Saving the gate_types of eadh gate as a dictionary for later use in the gate function
gate_type = {}
for node in pi:
     gate_type.update({node:"PI"})
for node in l:
     gate_type.update({node.output:node.g_type})
nx.set_node_attributes(g,gate_type,name = "gateType")
# Making a topologically sorted list of the nodes
nl = list(nx.topological_sort(g))


# Solving using the topological order

In [2]:
# Iterating through all the input sets given
for i in range(len(lineIn)-1):   

   # Iterating through the gates except pi
   for j in range(len(pi),len(nl)):

      # Finding the predecessors of the gate
      p = list(g.predecessors(nl[j]))
      out = 0 
      
      # Checking for all types of gates and performing the necessary operations
      if gate_type[nl[j]] == "nand2":
         out = int(not(int(value[i][p[0]]) and int(value[i][p[1]])))
    
      elif gate_type[nl[j]] == "and2":
         out = int(int(value[i][p[0]]) and int(value[i][p[1]]))

      elif gate_type[nl[j]] == "nor2":
         out = int(not(int(value[i][p[0]]) or int(value[i][p[1]])))
      
      elif gate_type[nl[j]] == "or2":
         out = int(int(value[i][p[0]]) or int(value[i][p[1]]))

      elif gate_type[nl[j]] == "inv":
         out = int(not(int(value[i][p[0]])))

      elif gate_type[nl[j]] == "xnor2":
         out = int(not(int(value[i][p[0]]) ^ int(value[i][p[1]])))

      elif gate_type[nl[j]] == "xor2":
         out = int(int(value[i][p[0]]) ^ int(value[i][p[1]]))
      
      # Updating the output of the current gate into the value list
      value[i].update({nl[j]:out})
           

# Using the Event - driven method

In [3]:
# Importing the
from queue import Queue

# Declaring and initializing the queue
Event = Queue()
for item in nl[:len(pi)]:
    Event.put(item)

# Iterating through all the input sets given
for i in range(len(lineIn)-1):
    
    # Iterating till the queue is empty
    while not Event.empty():
        e = Event.get()

        # Adding to the queue only if both its predessors are in list
        for thing in list(g.successors(e)):
            if list(g.predecessors(thing))[0] in value[i]:
                if list(g.predecessors(thing))[1] in value[i] :
                    Event.put(thing)
        # Adding to the dictionary if its not already there
        if e not in value1[i]:
            p = list(g.predecessors(e))

            # Going through all typed of gates to give the result
            if gate_type[e] == "nand2":
                out = int(not(int(value[i][p[0]]) and int(value[i][p[1]])))
                    
            if gate_type[e] == "and2":
                out = int(int(value[i][p[0]]) and int(value[i][p[1]]))

            elif gate_type[e] == "nor2":
                out = int(not(int(value[i][p[0]]) or int(value[i][p[1]])))
            
            elif gate_type[e] == "or2":
                out = int(int(value[i][p[0]]) or int(value[i][p[1]]))

            elif gate_type[e] == "inv":
                out = int(not(int(value[i][p[0]])))

            elif gate_type[e] == "xnor2":
                out = int(not(int(value[i][p[0]]) ^ int(value[i][p[1]])))

            elif gate_type[e] == "xor2":
                out = int(int(value[i][p[0]]) ^ int(value[i][p[1]]))    
                    
            value1[i].update({e:out})


IndexError: list index out of range

In [32]:
print(value[0])
print(value1[0])

{'N1': '0', 'N2': '1', 'N3': '0', 'N6': '0', 'N7': '0', 'n_0': 1, 'n_1': 1, 'n_3': 0, 'n_2': 1, 'N22': 1, 'N23': 1}
{'N1': '0', 'N2': '1', 'N3': '0', 'N6': '0', 'N7': '0', 'n_0': 1, 'n_1': 1, 'n_3': 0, 'n_2': 1, 'N22': 1, 'N23': 1}


In [31]:
for i in range(len(lineIn)-1):
    if value[i] != value1[i]:
        print("There are differences")

In [ ]:
from collections import deque

# Define the list N
N = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

# Define the size of the queue
n = 5

# Create a new deque object and add the first n values of N to the queue
queue = deque(N[:n])
l =[1,2,3]
queue.extend(l)
# Print the initial queue
print(queue)
if l in N:
    print("yes")

In [69]:
l=[1,2,3,4,5,6]
m = l[:]
m.append(1)
print(m)
print(l)

[1, 2, 3, 4, 5, 6, 1]
[1, 2, 3, 4, 5, 6]
